<a href="https://colab.research.google.com/github/fedorov/AI-Deep-Learning-Lab-2021/blob/idc-tcia/RSNA_2021_IDC_and_TCIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSNA 2021: Working with public datasets: TCIA and IDC

The goal of this session is to introduce you to the two data repositories supported by the US National Cancer Institute:

* The Cancer Imaging Archive (TCIA)
* Imaging Data Commons (IDC), which is the imaging repository within NCI Cancer Research Data Commons (CRDC)

**Learning Objectives:**
1. Understand basic capabilities of TCIA and IDC, and the differences between the two repositories.
2. Explore relevant functionality of TCIA and IDC to support data exploration, cohort definition, and retrieval of the data.
3. Learn how to analyze the data retrieved from TCIA/IDC on an example of a lung nodule segmentation task.

This notebook will guide you thought the complete process of identifying a relevant dataset, retrieving it, preparing it for processing by the specific analysis tool, installing the tool and applying it to the dataset, and visualizing the segmentation results produced by the tool.

Note that it is not the purpose of this tutorial to promote a specific tool, or assess its robustness. 

We aim to provide an example of how a tool can be used for analyzing a sample dataset from TCIA/IDC. We hope that after completing this tutorial you will be empowered and motivated to experiment with more tools and apply them to more datasets in TCIA/IDC!

**Session Authors**

* Andrey Fedorov
* Justin Kirby
* Dennis Bontempi

## Outline

1. Overview of the capabilities of the repositories
2. Installation of prerequisites
3. Lung nodule segmentation
4. Selection and download data from TCIA or IDC
4. Preprocessing of the data
5. Segmentation using nn-Unet
6. Preparation for the visualization of the results
6. Visualization of segmentation results
7. Exercises, next steps, getting help, references

TODO:
* cross link TCIA, IDC, nn-Unet...
* add references to publications



## Prerequisites

Make sure your Colab instance has a GPU! For this check "Runtime > Change runtime type" and make sure to choose the GPU runtime.

* google account
* Google cloud project with configured billing

## The Cancer Imaging Archive (TCIA)



## Imaging Data Commons (IDC)

## Lung nodule segmentation

For the sake of the example, we will focus on the problem of segmenting lung nodules, since there are numerous public collections that have data suitable for exploring this task, as well as various tools available. Notably, there are imaging collections that include manual segmentations of the nodules, which can be used to assess accuracy of automated analysis.

## Data selection

The model is trained to segment organs in chest CT. Let's identify some datasets that are suitable for using with this model.

### Selection and download of data from IDC

We will utilize IDC BigQuery SQL search interface to get the list of instances for the specific case we will use in the tutorial. 

You can also use portal, and we will have other examples of querying the data towards the end (Andrey TBD).

In [1]:
# TODO: need to discuss cloud prerequisites and project
from google.colab import auth
auth.authenticate_user()

In [ ]:
global my_PatientID = "LUNG1-002"
global my_CollectionID = "NSCLC-Radiomics"

In [ ]:
### %%bigquery --project=idc-tcia  --params={"patient_id":"LUNG1-002","collection_id":"NSCLC-Radiomics"} case002_df 


TODO: replace the below with BQ python client to simplify parameterization

In [62]:
%%bigquery --project=idc-tcia  --params={"patient_id":"100014","collection_id":"NLST"} case002_df 

WITH
  idc_manifest AS (
  SELECT
    PatientID,
    StudyInstanceUID,
    SeriesInstanceUID,
    SOPInstanceUID,
    Modality,
    SeriesDescription,
    gcs_url,
    collection_id as idc_collection_id
  FROM
    `canceridc-data.idc_current.dicom_all`
  WHERE
    PatientID = @patient_id
)
SELECT
  idc_manifest.*,
  # this is necessary since collection IDs used internally by IDC and TCIA are a bit different,
  # so we need to get the TCIA collection ID that will be recognized by TCIA API
  aux_table.tcia_api_collection_id
FROM
  idc_manifest
JOIN
  `canceridc-data.idc_current.auxiliary_metadata` AS aux_table
ON
  idc_manifest.SOPInstanceUID = aux_table.SOPInstanceUID
WHERE 
    # PatientID is unique and parameterization by collection_id is not really necessary,
    # but we use it here for consistency with the query we use with NBIA API, which does
    # require collection ID to be specified
    aux_table.tcia_api_collection_id = @collection_id


In [63]:
case002_df

,PatientID,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,Modality,SeriesDescription,gcs_url,idc_collection_id,tcia_api_collection_id
0,100014,1.2.840.113654.2.55.24837621142213487608020073...,1.2.840.113654.2.55.11312260835561211814968851...,1.2.840.113654.2.55.22165131652241387214537793...,CT,"1,OPA,SE,SEN16,B30f,314,2,120,40,32,na",gs://idc-open/aeaf1172-4740-4ecb-93f8-9137e02c...,nlst,NLST
1,100014,1.2.840.113654.2.55.24837621142213487608020073...,1.2.840.113654.2.55.11312260835561211814968851...,1.2.840.113654.2.55.25966844408640269564956948...,CT,"1,OPA,SE,SEN16,B30f,314,2,120,40,32,na",gs://idc-open/40056330-6cd2-4c47-ab73-0390e63f...,nlst,NLST
2,100014,1.2.840.113654.2.55.24837621142213487608020073...,1.2.840.113654.2.55.11312260835561211814968851...,1.2.840.113654.2.55.28931453271474684717696986...,CT,"1,OPA,SE,SEN16,B30f,314,2,120,40,32,na",gs://idc-open/1f2784f9-592a-420d-90b6-815f2c8e...,nlst,NLST
3,100014,1.2.840.113654.2.55.24837621142213487608020073...,1.2.840.113654.2.55.11312260835561211814968851...,1.2.840.113654.2.55.24024226320090963106361964...,CT,"1,OPA,SE,SEN16,B30f,314,2,120,40,32,na",gs://idc-open/d90a133f-5761-4844-948c-4665f016...,nlst,NLST
4,100014,1.2.840.113654.2.55.24837621142213487608020073...,1.2.840.113654.2.55.11312260835561211814968851...,1.2.840.113654.2.55.19956863316011512678250117...,CT,"1,OPA,SE,SEN16,B30f,314,2,120,40,32,na",gs://idc-open/d1da7759-187d-44a3-80bd-5e9bde4b...,nlst,NLST
...,...,...,...,...,...,...,...,...,...
460,100014,1.2.840.113654.2.55.24837621142213487608020073...,1.2.840.113654.2.55.11312260835561211814968851...,1.2.840.113654.2.55.30269068226283240818007390...,CT,"1,OPA,SE,SEN16,B30f,314,2,120,40,32,na",gs://idc-open/a3c7180a-9ba2-421b-a1c1-f67f42fa...,nlst,NLST
461,100014,1.2.840.113654.2.55.32681317118862120019139812...,1.2.840.113654.2.55.24212552127631641689348892...,1.2.840.113654.2.55.32357059180709804019649200...,CT,"2,OPA,SE,SEN16,B30f,297,2,120,40,32,na",gs://idc-open/bc77b19d-53d5-40fa-b69e-90c577fd...,nlst,NLST
462,100014,1.2.840.113654.2.55.23011183264553864936438136...,1.2.840.113654.2.55.10598059449409563834581839...,1.2.840.113654.2.55.44423699335405841753134292...,CT,"0,OPA,SE,SEN16,B30f,280,2,120,40,32,na",gs://idc-open/c7a445c7-8155-4b4e-9af1-2d0e1b8b...,nlst,NLST
463,100014,1.2.840.113654.2.55.23011183264553864936438136...,1.2.840.113654.2.55.10598059449409563834581839...,1.2.840.113654.2.55.21456577190815092306718770...,CT,"0,OPA,SE,SEN16,B30f,280,2,120,40,32,na",gs://idc-open/753aab9e-d276-45f1-91f8-d73e1c2f...,nlst,NLST


### Visualization of the selected study

TODO: explain that the data visualized is not the one here on VM, but in IDC and referenced by StudyInstanceUID

In [64]:
def get_idc_viewer_url(studyUID):
  return "https://viewer.imaging.datacommons.cancer.gov/viewer/"+studyUID

study_uid = case002_df["StudyInstanceUID"].unique()[0]
print(get_idc_viewer_url(study_uid))

https://viewer.imaging.datacommons.cancer.gov/viewer/1.2.840.113654.2.55.248376211422134876080200734633596809407


In [65]:
import pandas as pd
case002_df['SeriesInstanceUID'].unique()

array(['1.2.840.113654.2.55.113122608355612118149688511074855524649',
       '1.2.840.113654.2.55.242125521276316416893488922442135031827',
       '1.2.840.113654.2.55.105980594494095638345818398307575745148',
       '1.2.840.113654.2.55.224370998422513161237132457653669305128',
       '1.2.840.113654.2.55.335578899304552476938977231801665435563',
       '1.2.840.113654.2.55.115865465855988191201389124609998041309'],
      dtype=object)

In [69]:
pd.set_option('display.max_colwidth', None)
case002_df.groupby(['StudyInstanceUID','SeriesInstanceUID','SeriesDescription','Modality']).size().reset_index().rename(columns={0:'count'})


,StudyInstanceUID,SeriesInstanceUID,SeriesDescription,Modality,count
0,1.2.840.113654.2.55.230111832645538649364381369281963718932,1.2.840.113654.2.55.105980594494095638345818398307575745148,"0,OPA,SE,SEN16,B30f,280,2,120,40,32,na",CT,153
1,1.2.840.113654.2.55.230111832645538649364381369281963718932,1.2.840.113654.2.55.335578899304552476938977231801665435563,"0,OPL,SE,SEN16,T20s,512,na,120,195.5,195,na",CT,2
2,1.2.840.113654.2.55.248376211422134876080200734633596809407,1.2.840.113654.2.55.113122608355612118149688511074855524649,"1,OPA,SE,SEN16,B30f,314,2,120,40,32,na",CT,150
3,1.2.840.113654.2.55.248376211422134876080200734633596809407,1.2.840.113654.2.55.224370998422513161237132457653669305128,"1,OPL,SE,SEN16,T20s,512,na,80,208,208,na",CT,2
4,1.2.840.113654.2.55.326813171188621200191398126261396586729,1.2.840.113654.2.55.115865465855988191201389124609998041309,"2,OPL,SE,SEN16,T20s,512,na,80,205.6,205,na",CT,1
5,1.2.840.113654.2.55.326813171188621200191398126261396586729,1.2.840.113654.2.55.242125521276316416893488922442135031827,"2,OPA,SE,SEN16,B30f,297,2,120,40,32,na",CT,157


In [70]:
# CHANGE ME!
StudyInstanceUID = "1.2.840.113654.2.55.248376211422134876080200734633596809407"

In [76]:
import os
idc_download_folder = "/content/IDC_downloads"
if not os.path.exists(idc_download_folder):
  os.mkdir(idc_download_folder)

case002_manifest = os.path.join(idc_download_folder, "idc_manifest.txt")
case002_df[case002_df["StudyInstanceUID"]==StudyInstanceUID]["gcs_url"].to_csv(case002_manifest, header=False, index=False)

In [74]:
!cat /content/IDC_downloads/idc_manifest.txt |wc

    152     152    8360


In [ ]:
%%capture
!rm -rf /content/IDC_downloads/*.dcm
!cat /content/IDC_downloads/idc_manifest.txt | gsutil -u idc-tcia -m cp -I /content/IDC_downloads

### Selection and download of data from TCIA

For the sake of simplicity, we will download images for a specific case that we know has CT of the chest, and segmentations of the organs of interest. You can utilize TCIA portal, or other features of TCIA API to select data (Justin TBD what should be discussed here).



In [78]:
!rm -rf /content/TCIA_downloads/*.dcm

In [79]:
import requests
params = {"Collection":"NSCLC-Radiomics", "PatientID":"LUNG1-002"}
r = requests.get("https://services.cancerimagingarchive.net/nbia-api/services/v1/getSeries", params=params)
if r.status_code == 200:
  df = pd.read_json(r.text)
else:
  print(f"Failed with {r.status_code}")

In [80]:
df

,SeriesInstanceUID,StudyInstanceUID,Modality,BodyPartExamined,SeriesNumber,Collection,PatientID,Manufacturer,ManufacturerModelName,SoftwareVersions,ImageCount,TimeStamp,SeriesDate,SeriesDescription
0,1.3.6.1.4.1.32722.99.99.232988001551799080335895423941323261228,1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095,CT,LUNG,1,NSCLC-Radiomics,LUNG1-002,"CMS, Inc.",XiO,4.34.02,111,2020-10-07 14:01:19,NaN,NaN
1,1.3.6.1.4.1.32722.99.99.243267551266911245830259417117543245931,1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095,RTSTRUCT,NaN,4,NSCLC-Radiomics,LUNG1-002,Varian Medical Systems,ARIA RadOnc,15.5.11,1,2020-10-08 14:18:10,NaN,NaN
2,1.2.276.0.7230010.3.1.3.2323910823.11504.1597260515.421,1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095,SEG,LUNG,300,NSCLC-Radiomics,LUNG1-002,QIICR,https://github.com/qiicr/dcmqi.git,3efde87,1,2020-08-20 18:38:20,2020-08-12 00:00:00.0,Segmentation


In [ ]:
import json, zipfile
series_list = [item["SeriesInstanceUID"] for item in json.loads(r.text)]
print(series_list)

tcia_download_folder = "/content/TCIA_downloads"
if not os.path.exists(tcia_download_folder):
  os.mkdir(tcia_download_folder)

for series_uid in series_list:

  # download zip file with the series instances
  params = {"SeriesInstanceUID":series_uid}
  image_request = requests.get(" https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage", params=params, stream=True)
  print(f"Completed request: {image_request.url}")
  if image_request.status_code == 200:
    series_zip_name = os.path.join(tcia_download_folder, f"{series_uid}.zip")
    with open(series_zip_name, "wb") as f:
      for chunk in image_request.iter_content(chunk_size=1024):
        f.write(chunk)
    print(f"Downloaded and saved series {series_uid}")

    # extract individual instances from the series zip file
    series_folder_name = os.path.join(tcia_download_folder, series_uid)
    if not os.path.exists(series_folder_name):
      os.mkdir(series_folder_name)
    with zipfile.ZipFile(series_zip_name, 'r') as zip_ref:
      zip_ref.extractall(series_folder_name)
  else:
    print(f"Failed with {r.status_code}")

['1.3.6.1.4.1.32722.99.99.232988001551799080335895423941323261228', '1.3.6.1.4.1.32722.99.99.243267551266911245830259417117543245931', '1.2.276.0.7230010.3.1.3.2323910823.11504.1597260515.421']
Completed request: https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage?SeriesInstanceUID=1.3.6.1.4.1.32722.99.99.232988001551799080335895423941323261228
Downloaded and saved series 1.3.6.1.4.1.32722.99.99.232988001551799080335895423941323261228
Completed request: https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage?SeriesInstanceUID=1.3.6.1.4.1.32722.99.99.243267551266911245830259417117543245931
Downloaded and saved series 1.3.6.1.4.1.32722.99.99.243267551266911245830259417117543245931
Completed request: https://services.cancerimagingarchive.net/nbia-api/services/v1/getImage?SeriesInstanceUID=1.2.276.0.7230010.3.1.3.2323910823.11504.1597260515.421
Downloaded and saved series 1.2.276.0.7230010.3.1.3.2323910823.11504.1597260515.421


In [25]:
!git clone https://github.com/pieper/dicomsort.git
!pip install pydicom
!python dicomsort/dicomsort.py --help

Cloning into 'dicomsort'...
remote: Enumerating objects: 126, done.
remote: Total 126 (delta 0), reused 0 (delta 0), pack-reused 126
Receiving objects: 100% (126/126), 37.03 KiB | 9.26 MiB/s, done.
Resolving deltas: 100% (63/63), done.

% dicomsort.py --help
dicomsort [options...] sourceDir targetDir/<patterns>

 where [options...] can be:
    [-z,--compressTargets] - create a .zip file in the target directory
    [-d,--deleteSource] - remove source files/directories after sorting
    [-f,--forceDelete] - remove source without confirmation
    [-k,--keepGoing] - report but ignore dupicate target files
    [-v,--verbose] - print diagnostics while processing
    [-s,--symlink] - create a symlink to dicom files in sourceDir instead of copying them
    [-t,--test] - run the built in self test (requires internet)
    [-u,--unsafe] - do not replace unsafe characters with '_' in the path
    [--help] - print this message

 where sourceDir is directory to be scanned or "" (null string) to read

In [26]:
!mkdir -p IDC_sorted && mkdir -p TCIA_sorted
!python dicomsort/dicomsort.py -k -u IDC_downloads IDC_sorted/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm
!python dicomsort/dicomsort.py -k -u TCIA_downloads TCIA_sorted/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm

100% 114/114 [00:02<00:00, 40.55it/s]
Files sorted
Source directory does not exist: TCIA_downloads


### Download consistency check

Let's check the content downloaded from the two repositories is identical, just in case.

In [ ]:
!diff -r IDC_sorted TCIA_sorted

Only in IDC_sorted: LUNG1-002.json


In [27]:
!mkdir -p tutorial/data/dicom && mv IDC_sorted/* tutorial/data/dicom

## Setup of the Colab VM



In the following cells we will confirm you have a GPU before doing anything else, and will install and import all the Python dependencies. 

The main python packages we need to install are:
* `nnunet` - which is the [codebase for the nn-UNet framework](https://github.com/MIC-DKFZ/nnUNet) we are going to be using for the segmentation step;
* `pydicom`, a Python [package](https://github.com/pydicom/pydicom) that lets the use read, modify, and write DICOM data in an easy "pythonic" way - that we are going to use to distinguish different DICOM objects from each other.

### GPU checks

In [11]:
# check wether the Colab Instance was correctly changed to a GPU instance
gpu_list = !nvidia-smi --list-gpus

has_gpu = False if "failed" in gpu_list[0] else True

if not has_gpu:
  print("Your Colab VM does not have a GPU - check \"Runtime > Change runtime type\"")

In [12]:
# check which model of GPU the notebook is equipped with - a Tesla K80 or T4
# T4 is the best performing on the two - and can about half the GPU processing time

!nvidia-smi

Mon Nov 15 19:00:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Python packages

In [13]:
%%capture
!pip install nnunet
!pip install pydicom

In [14]:
import os
import sys
import shutil

import time
import gdown

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk

from medpy.metric.binary import dc as dice_coef
from medpy.metric.binary import hd as hausdorff_distance
from medpy.metric.binary import asd as avg_surf_distance

from medpy.filter.binary import largest_connected_component

# use the "tensorflow_version" magic to make sure TF 1.x is imported
%tensorflow_version 1.x
import tensorflow as tf
import keras

print("Python version               : ", sys.version.split('\n')[0])
print("Numpy version                : ", np.__version__)
print("TensorFlow version           : ", tf.__version__)
print("Keras (stand-alone) version  : ", keras.__version__)

print("\nThis Colab instance is equipped with a GPU.")

# ----------------------------------------

#everything that has to do with plotting goes here below
import matplotlib
matplotlib.use("agg")

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline
%config InlineBackend.figure_format = "png"

import ipywidgets as ipyw

## ----------------------------------------

# create new colormap appending the alpha channel to the selected one
# (so that we don't get a \"color overlay\" when plotting the segmask superimposed to the CT)
cmap = plt.cm.Reds
my_reds = cmap(np.arange(cmap.N))
my_reds[:,-1] = np.linspace(0, 1, cmap.N)
my_reds = ListedColormap(my_reds)

cmap = plt.cm.Greens
my_greens = cmap(np.arange(cmap.N))
my_greens[:,-1] = np.linspace(0, 1, cmap.N)
my_greens = ListedColormap(my_greens)

cmap = plt.cm.Blues
my_blues = cmap(np.arange(cmap.N))
my_blues[:,-1] = np.linspace(0, 1, cmap.N)
my_blues = ListedColormap(my_blues)

cmap = plt.cm.spring
my_spring = cmap(np.arange(cmap.N))
my_spring[:,-1] = np.linspace(0, 1, cmap.N)
my_spring = ListedColormap(my_spring)
## ----------------------------------------

import seaborn as sns

TensorFlow 1.x selected.
Python version               :  3.7.12 (default, Sep 10 2021, 00:21:48) 
Numpy version                :  1.19.5
TensorFlow version           :  1.15.2
Keras (stand-alone) version  :  2.3.1

This Colab instance is equipped with a GPU.


Using TensorFlow backend.


In [46]:
# PyPlastimatch - python wrapper for Plastimatch (and interactive notebook visualisation)
!svn checkout https://github.com/AIM-Harvard/pyplastimatch/trunk/pyplastimatch tutorial/pyplastimatch

svn: E155000: '/content/tutorial/pyplastimatch' is already a working copy for a different URL


### Command-line tools


The only package we will need for this tutorial is [Plastimatch](https://plastimatch.org/index.html). Plastimatch is considered to be the swiss army knife of medical images processing: we will use it to convert DICOM (CT, RTSTRUCT) series to NRRD files - but it can be used for a multitude of other tasks, such as registration, resampling, cropping, and computing statistics to name a few. Plastimatch is also available as a 3DSlicer plug-in and can be used directly from the Slicer GUI.

For the sake of clarity and simplicity, we will call Plastimatch from a very simple [Python wrapper](https://github.com/denbonte/pyplastimatch) written for the occasion (unfortunately, Plastimatch does not provide an official one) - more on this later.

In [15]:
%%capture
!sudo apt update

!sudo apt install plastimatch

In [ ]:
!echo $(plastimatch --version)

plastimatch version 1.7.0


We are also going to install subversion, a tool that will allow us to clone GitHub repositories only partially (to save time and space).

In [16]:
%%capture

!sudo apt install subversion

In [ ]:
!echo $(svn --version | head -n 2)

svn, version 1.9.7 (r1800392) compiled Mar 28 2018, 08:49:13 on x86_64-pc-linux-gnu


### Environment Setup

Set the Linux environment variables needed to run the nnU-Net pipeline. 

Three main variables are needed by default to run the nnU-Net segmentation pipelines:
* `nnUNet_raw_data_base` is the path to the folder where the segmentation pipeline expects to find the data to process;
* `nnUNet_preprocessed` is the path to the folder where the preprocessed data are saved;
* `RESULTS_FOLDER` is the path to the folder storing by default the model weights and, in our case, for simplicity, the segmentation masks produced by the pipeline.

We will use the additional variable `PATH_TO_MODEL_FILE` to point to the location where the pre-trained model weights for the chosen model will be stored (more on this later).

Please notice that these variables need to be set using `os.environ[]` in Google Colab - as `!export` is not sufficient to guarantee the variables are kept from one cell to the other. For more in-depth information regarding what the nnU-Net framework uses these folders for, please visit [the dedicated nnU-Net documentation page](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md)

In [17]:
# set env variables for the bash process
import os
os.environ['nnUNet_raw_data_base'] = "/content/tutorial/data/nnUNet_raw_data/"
os.environ['nnUNet_preprocessed'] = "/content/tutorial/data/processed/"

os.environ["RESULTS_FOLDER"] = "/content/tutorial/output/"
os.environ["PATH_TO_MODEL_FILE"] = "/content/tutorial/models/Task055_SegTHOR.zip"

## nn-UNet model setup

### Thoracic Organs at Risk Segmentation AI Model

While the nnU-Net framework should take care of the model download (from Zenodo), some of the zip files containing the pre-trained weights are particularly large, so the download can take a lot of time, get stuck, or produce errors (as [reported by other users](https://github.com/MIC-DKFZ/nnUNet/issues/358#issue-726410474) and in the [repository FAQ](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/common_problems_and_solutions.md#downloading-pretrained-models-unzip-cannot-find-zipfile-directory-in-one-of-homeisenseennunetdownload_16031094034174126)) .

For this reason, and for the purpose of speeding up this tutorial, we decided to copy the relevant model weights in a shared Dropbox folder. In the following cells, we use Linux `wget` to pull such files from the folder - and exploit the nnU-Net framework command `nnUNet_install_pretrained_model_from_zip` to unpack and install the pre-trained model.

In [18]:
# create the directory tree
!mkdir -p tutorial 
!mkdir -p tutorial/models tutorial/data tutorial/output
!mkdir -p tutorial/data/dicom tutorial/data/processed tutorial/data/nnUNet_raw_data

In [19]:
# this will usually take between one and five minutes (but can sometimes take up to eight)
seg_model_url = "https://www.dropbox.com/s/m7es2ojn8h0ybhv/Task055_SegTHOR.zip?dl=0"
output_path = "tutorial/models/Task055_SegTHOR.zip"

!wget -O $output_path $seg_model_url

--2021-11-15 19:14:37--  https://www.dropbox.com/s/m7es2ojn8h0ybhv/Task055_SegTHOR.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.68.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.68.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/m7es2ojn8h0ybhv/Task055_SegTHOR.zip [following]
--2021-11-15 19:14:37--  https://www.dropbox.com/s/raw/m7es2ojn8h0ybhv/Task055_SegTHOR.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6b8200b0308203dd2f29eff6c2.dl.dropboxusercontent.com/cd/0/inline/BaDwWWqCFxYRuQmpLKCdihbbhlBz7_eZJ7BBYtxBTba5f76VsBVPu3yx7f6aMW3ZuSuWgedhojWeYXL3cxvEW9PhuI8yn-XTD9Rzt6ot6_KOBLu7gKrCL5ZvxjBOjd71pv_Wcn33vBIhAJrCbSs499Hc/file# [following]
--2021-11-15 19:14:37--  https://uc6b8200b0308203dd2f29eff6c2.dl.dropboxusercontent.com/cd/0/inline/BaDwWWqCFxYRuQmpLKCdihbbhlBz7_eZJ7BBYtxBTba5f76VsBVPu3yx7f6aM

Unpack and install model (under `PATH_TO_MODEL_FILE`).

In [37]:
%%capture
!nnUNet_install_pretrained_model_from_zip $PATH_TO_MODEL_FILE

## Data Pre-processing

In order to run the AI segmentation pipeline, we need to convert the DICOM data in a format which python can easily handle

Using the simple Plastimatch wrapper, let's convert the DICOM CT series in both NRRD (very flexible, simple handling with SimpleITK) and NIfTI (as required by the nnU-Net pipeline) format. Furthermore, we are going to convert the DICOM RTSTRUCT Series into a NRRD volume as well - to allow for comparison between the manual and automatic segmentation, later in the notebook.


In [22]:
from tutorial.pyplastimatch import pyplastimatch as pypla
from tutorial.pyplastimatch.utils import viz as viz_utils
from tutorial.pyplastimatch.utils import data as data_utils

In [31]:
pat = "LUNG1-002"
StudyInstanceUID = case002_df["StudyInstanceUID"].unique()[0]
SeriesInstanceUID_ct = case002_df[case002_df["Modality"]=="CT"]["SeriesInstanceUID"].unique()[0]
SeriesInstanceUID_rt = case002_df[case002_df["Modality"]=="RTSTRUCT"]["SeriesInstanceUID"].unique()[0]

# directories storing NRRD and NIfTI files
base_preproc_path = "tutorial/data/processed"

pat_dir_path_nrrd = os.path.join(base_preproc_path, "nrrd", pat)
pat_dir_path_nii = os.path.join(base_preproc_path, "nii", pat)
  
# patient subfolder where all the preprocessed NRRDs will be stored
if not os.path.exists(pat_dir_path_nrrd): os.makedirs(pat_dir_path_nrrd)
  
# patient subfolder where all the preprocessed NIfTIs will be stored
if not os.path.exists(pat_dir_path_nii): os.makedirs(pat_dir_path_nii)

# path to the directory where the DICOM CT file is stored
path_to_ct_dir = os.path.join("tutorial/data/dicom", pat,
                              StudyInstanceUID, SeriesInstanceUID_ct)

# path to the files where the NRRD and NIfTI CTs will be stored
ct_nrrd_path = os.path.join(pat_dir_path_nrrd, pat + "_ct.nrrd")
ct_nii_path = os.path.join(pat_dir_path_nii, pat + "_ct.nii.gz")


# path to the directory where the DICOM RTSTRUCT file is stored
path_to_rt_dir = os.path.join("tutorial/data/dicom", pat,
                              StudyInstanceUID, SeriesInstanceUID_rt)

# path to the files where the NRRD RTSTRUCTs will be stored
rt_folder = os.path.join(pat_dir_path_nrrd, "RTSTRUCT")
rt_list_path = os.path.join(pat_dir_path_nrrd, "RTSTRUCT_content")


verbose = True

# logfile for the plastimatch conversion
log_file_path_nrrd = os.path.join(pat_dir_path_nrrd, pat + '_pypla.log')
log_file_path_nii = os.path.join(pat_dir_path_nii, pat + '_pypla.log')
  
# DICOM CT to NRRD conversion (if the file doesn't exist yet)
if not os.path.exists(ct_nrrd_path):
  convert_args_ct = {"input" : path_to_ct_dir,
                     "output-img" : ct_nrrd_path}
  
  # clean old log file if it exist
  if os.path.exists(log_file_path_nrrd): os.remove(log_file_path_nrrd)
  
  pypla.convert(verbose = verbose, path_to_log_file = log_file_path_nrrd, **convert_args_ct)

# DICOM RTSTRUCT to NRRD conversion (if the file doesn't exist yet)
if not os.path.exists(rt_folder):
  convert_args_rt = {"input" : path_to_rt_dir, 
                     "referenced-ct" : path_to_ct_dir,
                     "output-prefix" : rt_folder,
                     "prefix-format" : 'nrrd',
                     "output-ss-list" : rt_list_path}
  
  # clean old log file if it exist
  if os.path.exists(log_file_path_nrrd): os.remove(log_file_path_nrrd)
  
  pypla.convert(verbose = verbose, path_to_log_file = log_file_path_nrrd, **convert_args_rt)

# DICOM CT to NIfTI conversion (if the file doesn't exist yet)
if not os.path.exists(ct_nii_path):
  convert_args_nii = {"input" : path_to_ct_dir, 
                      "output-img" : ct_nii_path}
  
  # clean old log file if it exist
  if os.path.exists(log_file_path_nii): os.remove(log_file_path_nii)
  
  pypla.convert(verbose = verbose, path_to_log_file = log_file_path_nii, **convert_args_nii)


Running 'plastimatch convert' with the specified arguments:
  --input tutorial/data/dicom/LUNG1-002/1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095/1.3.6.1.4.1.32722.99.99.232988001551799080335895423941323261228
  --output-img tutorial/data/processed/nrrd/LUNG1-002/LUNG1-002_ct.nrrd
... Done.

Running 'plastimatch convert' with the specified arguments:
  --input tutorial/data/dicom/LUNG1-002/1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095/1.3.6.1.4.1.32722.99.99.243267551266911245830259417117543245931
  --referenced-ct tutorial/data/dicom/LUNG1-002/1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095/1.3.6.1.4.1.32722.99.99.232988001551799080335895423941323261228
  --output-prefix tutorial/data/processed/nrrd/LUNG1-002/RTSTRUCT
  --prefix-format nrrd
  --output-ss-list tutorial/data/processed/nrrd/LUNG1-002/RTSTRUCT_content
... Done.

Running 'plastimatch convert' with the specified arguments:
  --input tutorial/data/dicom/LUNG1-002/1.3.6.1.4

As a last step before running the lung nodules segmentation pipeline, we need to make sure the folder storing the data follows the structure required by the nnU-Net framework, described at the [dedicated documentation page](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/dataset_conversion.md).

In [32]:
# create a folder (random task name) for nnU-Net inference
proc_folder_path = os.path.join(os.environ["nnUNet_raw_data_base"],
                                "segthor", "imagesTs")

!mkdir -p $proc_folder_path

# populate the folder following the nnU-Net naming conventions
copy_path = os.path.join(proc_folder_path, pat + "_0000.nii.gz")

# copy NIfTI to the right dir for nnU-Net processing
if not os.path.exists(copy_path):
  shutil.copy(src = ct_nii_path, dst = copy_path)

## Segmentation of thoracic structures from CT series

### Inference 

In order to run the segmentation pipeline, we can follow the ["run inference" section of the nnU-Net documentation](https://github.com/MIC-DKFZ/nnUNet#how-to-run-inference-with-pretrained-models), specifying the path to the input and output folders defined in the sections above, and the pretrained model we want to use (i.e., the one we downloaded earlier).

For the purpose of this notebook, to make the processing faster, we are not going to use an ensemble of different U-Net configurations for inference or test time augmentation (TTA). You are invited to explore these options later - and if you decide to do so, you can read [this example](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/inference_example_Prostate.md) from the nnU-Net documentation to learn how this can be achieved.

To learn more about all the arguments that can be specified to the `nnUNet_predict` command, run `nnUNet_predict --help`.

In [34]:
!echo $RESULTS_FOLDER

/content/tutorial/output/


In [35]:
!mkdir -p /content/tutorial/output/nnUNet/2d/Task055_SegTHOR/nnUNetTrainerV2__nnUNetPlansv2.1

In [38]:
# run the inference phase
# accepted options for --model are: 2d, 3d_lowres, 3d_fullres or 3d_cascade_fullres
!nnUNet_predict --input_folder "tutorial/data/nnUNet_raw_data/segthor/imagesTs" \
                --output_folder $RESULTS_FOLDER \
                --task_name "Task055_SegTHOR" --model 2d --disable_tta 



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /content/tutorial/output/nnUNet/2d/Task055_SegTHOR/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 1 input modalities for each image
Found 1 unique case ids, here are some examples: ['LUNG1-002']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 1
number of cases that still need to be predicted: 1
emptying cuda cache
loading parameters for folds, None
folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds:  ['/content/tutorial/output/nnUNet/2d/Task055_SegTHOR/nnUNetTrainerV2__nnUNet

### Post-processing of inference results

After the inference is finished, we can convert the segmentation masks back to NRRD for visualisation purposes and for easier handling.

In [39]:
pred_nii_path = os.path.join(os.environ["RESULTS_FOLDER"], pat + ".nii.gz")

sitk_ct = sitk.ReadImage(ct_nrrd_path)

nrrd_spacing = sitk_ct.GetSpacing()
nrrd_dim = sitk_ct.GetSize()

nii_spacing = tuple(nib.load(pred_nii_path).header['pixdim'][1:4])
nii_dim = tuple(nib.load(pred_nii_path).get_fdata().shape)

assert (nrrd_spacing == nii_spacing) & (nrrd_dim == nii_dim)

## ----------------------------------------
# NIfTI TO NRRD CONVERSION

# path to the output NRRD file (inferred segmasks)
pred_nrrd_path = os.path.join(pat_dir_path_nrrd, pat + "_pred_segthor.nrrd")
log_file_path = os.path.join(pat_dir_path_nrrd, pat + "_pypla.log")

# Inferred NIfTI segmask to NRRD
convert_args_pred = {"input" : pred_nii_path, 
                     "output-img" : pred_nrrd_path}

pypla.convert(path_to_log_file = log_file_path, **convert_args_pred)


Running 'plastimatch convert' with the specified arguments:
  --input /content/tutorial/output/LUNG1-002.nii.gz
  --output-img tutorial/data/processed/nrrd/LUNG1-002/LUNG1-002_pred_segthor.nrrd
... Done.


### Visualising the Segmentation Masks

We can visualise the raw AI-inferred segmentation mask (heart, aorta, esophagus, amd treachea - in green, yellow, red, and blue, respectively) and compare the heart (and esophagus, if available for the randomly selected patient) segmentation to the manual delineation.

In [40]:
# load NRRD volumes
ct_nrrd = sitk.GetArrayFromImage(sitk_ct)

# inferred segmask
pred_nrrd_segthor = sitk.GetArrayFromImage(sitk.ReadImage(pred_nrrd_path))

pred_nrrd_esophagus = np.copy(pred_nrrd_segthor)
pred_nrrd_heart = np.copy(pred_nrrd_segthor)
pred_nrrd_trachea = np.copy(pred_nrrd_segthor)
pred_nrrd_aorta = np.copy(pred_nrrd_segthor)
  
# zero every segmask other than the esophagus and make the mask binary (0/1)
pred_nrrd_esophagus[pred_nrrd_segthor != 1] = 0
pred_nrrd_esophagus[pred_nrrd_esophagus != 0] = 1
  
# zero every segmask other than the heart and make the mask binary (0/1)
pred_nrrd_heart[pred_nrrd_segthor != 2] = 0
pred_nrrd_heart[pred_nrrd_heart != 0] = 1
  
# zero every segmask other than the trachea and make the mask binary (0/1)
pred_nrrd_trachea[pred_nrrd_segthor != 3] = 0
pred_nrrd_trachea[pred_nrrd_trachea != 0] = 1
  
# zero every segmask other than the aorta and make the mask binary (0/1)
pred_nrrd_aorta[pred_nrrd_segthor != 4] = 0
pred_nrrd_aorta[pred_nrrd_aorta != 0] = 1


# manual segmask (from the RTSTRUCT)
rt_segmask_heart = os.path.join(pat_dir_path_nrrd, "RTSTRUCT", "Heart.nrrd")
rt_nrrd_heart = sitk.GetArrayFromImage(sitk.ReadImage(rt_segmask_heart))

try:
  rt_segmask_esophagus = os.path.join(pat_dir_path_nrrd, "RTSTRUCT", "Esophagus.nrrd")
  rt_nrrd_esophagus = sitk.GetArrayFromImage(sitk.ReadImage(rt_segmask_esophagus))
except:
  # for the sake of simplicity, fill the volume with zeros
  # (so that we can keep the code that comes after the same)
  rt_nrrd_esophagus = np.zeros(rt_nrrd_heart.shape)

In [41]:
_ = viz_utils.AxialSliceSegmaskComparison(ct_volume = ct_nrrd,
                                          segmask_ai_dict = {"Heart" : pred_nrrd_heart,
                                                             "Aorta" : pred_nrrd_aorta,
                                                             "Trachea" : pred_nrrd_trachea,
                                                             "Esophagus" : pred_nrrd_esophagus},
                                          segmask_manual_dict = {"Heart" : rt_nrrd_heart,
                                                                 "Esophagus" : rt_nrrd_esophagus},
                                          segmask_cmap_dict = {"Heart" : my_greens,
                                                               "Aorta" : my_spring,
                                                               "Esophagus" : my_reds,
                                                               "Trachea" : my_blues},
                                          dpi = 100)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [48]:
_ = viz_utils.AxialSliceSegmaskViz(ct_volume = ct_nrrd,
                                          segmask_dict = {"Heart" : pred_nrrd_heart,
                                                             "Aorta" : pred_nrrd_aorta,
                                                             "Trachea" : pred_nrrd_trachea,
                                                             "Esophagus" : pred_nrrd_esophagus},
                                          segmask_cmap_dict = {"Heart" : my_greens,
                                                               "Aorta" : my_spring,
                                                               "Esophagus" : my_reds,
                                                               "Trachea" : my_blues},
                                          dpi = 100)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

### Quantitative assessment of the results

Let's start by defining a function to compute the center of mass (CoM) of the segmentation masks. Before computing the common segmentation metrics, the CoM can give us a rough idea of how different the 3D delineations are and if there are any major labelling errors (which we could correct, e.g., with a largest connected component analysis).

We will base our function on the [implementation](https://github.com/AIM-Harvard/pyradiomics/blob/master/radiomics/generalinfo.py) found in the open source [PyRadiomics library](https://github.com/AIM-Harvard/pyradiomics).

In [42]:
def getCenterOfMassIndexValue(input_mask):
    
    """
    Returns z, y and x coordinates of the center of mass of the ROI in terms of
    the image coordinate space (continuous index).

    Calculation is based on the original (non-resampled) mask.
    Because this represents the continuous index, the order of x, y and z is reversed,
    i.e. the first element is the z index, the second the y index and the last element is the x index.

    @params:
      input_mask - required : numpy (binary) volume storing the segmentation mask.

    """

    if input_mask is not None:
      mask_coordinates = np.array(np.where(input_mask == 1))
      center_index = np.mean(mask_coordinates, axis = 1)
      return tuple(center_index)
    else:
      return None

In [43]:
com_manual_heart = np.array(getCenterOfMassIndexValue(rt_nrrd_heart))
com_manual_heart_int = np.ceil(com_manual_heart).astype(dtype = np.uint16)

com_raw_heart = np.array(getCenterOfMassIndexValue(pred_nrrd_heart))
com_raw_heart_int = np.ceil(com_raw_heart).astype(dtype = np.uint16)

print("Heart Center of Mass (raw AI segmentation) \t:", com_raw_heart_int)
print("Heart Center of Mass (manual segmentation) \t:", com_manual_heart_int)

Heart Center of Mass (raw AI segmentation) 	: [ 37 210 287]
Heart Center of Mass (manual segmentation) 	: [ 34 208 292]


In [44]:
com_manual_heart = np.array(getCenterOfMassIndexValue(rt_nrrd_heart))
com_manual_heart_int = np.ceil(com_manual_heart).astype(dtype = np.uint16)

com_raw_heart = np.array(getCenterOfMassIndexValue(pred_nrrd_heart))
com_raw_heart_int = np.ceil(com_raw_heart).astype(dtype = np.uint16)

print("Heart Center of Mass (raw AI segmentation) \t:", com_raw_heart_int)
print("Heart Center of Mass (manual segmentation) \t:", com_manual_heart_int)

## ----------------------------------------

# run this if and only if a manual esophagus segmentation mask is available
if np.sum(rt_nrrd_esophagus):

  com_manual_esophagus = np.array(getCenterOfMassIndexValue(rt_nrrd_esophagus))
  com_manual_esophagus_int = np.ceil(com_manual_esophagus).astype(dtype = np.uint16)

  com_raw_esophagus = np.array(getCenterOfMassIndexValue(pred_nrrd_esophagus))
  com_raw_esophagus_int = np.ceil(com_raw_esophagus).astype(dtype = np.uint16)

  print("\nEsophagus Center of Mass (raw AI segmentation) \t:", com_raw_esophagus_int)
  print("Esophagus Center of Mass (manual segmentation) \t:", com_manual_esophagus_int)

Heart Center of Mass (raw AI segmentation) 	: [ 37 210 287]
Heart Center of Mass (manual segmentation) 	: [ 34 208 292]

Esophagus Center of Mass (raw AI segmentation) 	: [ 58 260 263]
Esophagus Center of Mass (manual segmentation) 	: [ 55 255 267]


Another common way to evaluate the quality of the segmentation is computing the Dice Coefficient between the AI segmentation and the manual one. To do so, we will use [MedPy's implementation of the Dice coefficient](https://loli.github.io/medpy/generated/medpy.metric.binary.dc.html#medpy-metric-binary-dc) (for binary masks).

We can use other MedPy's functions to compute the Hausdorff distance and the average surface distance as well*.

_*in most cases, the Hausdorff Distance will be quite high for both the heart segmentation and, if available with the randomly selected patient, the esophagus one. This is not a clear indication the model performance is poor: rather, it could also be the segmentation guidelines of the two datasets (the one the nnU-Net model was trained on and the external and independent validation dataset pulled from IDC) differ significantly._

In [45]:
pred_nrrd_path = os.path.join(pat_dir_path_nrrd, pat + "_pred_segthor.nrrd")


voxel_spacing = list(sitk_ct.GetSpacing())

dc_heart = dice_coef(pred_nrrd_heart, rt_nrrd_heart)
hd_heart = hausdorff_distance(pred_nrrd_heart, rt_nrrd_heart, voxelspacing = voxel_spacing)
asd_heart = avg_surf_distance(pred_nrrd_heart, rt_nrrd_heart, voxelspacing = voxel_spacing)

print("Heart Dice Coefficient (raw segmentation) :", dc_heart)
print("Heart Hausdorff Distance (raw segmentation) [mm]:", hd_heart)
print("Heart Average Surface Distance (raw segmentation) [mm]:", asd_heart)


# run this if and only if a manual esophagus segmentation mask is available
if np.sum(rt_nrrd_esophagus):
  dc_esophagus = dice_coef(pred_nrrd_esophagus, rt_nrrd_esophagus)
  hd_esophagus = hausdorff_distance(pred_nrrd_esophagus, rt_nrrd_esophagus, voxelspacing = voxel_spacing)
  asd_esophagus = avg_surf_distance(pred_nrrd_esophagus, rt_nrrd_esophagus, voxelspacing = voxel_spacing)

  print("\nEsophagus Dice Coefficient (raw segmentation) :", dc_esophagus)
  print("Esophagus Hausdorff Distance (raw segmentation) [mm]:", hd_esophagus)
  print("Esophagus Average Surface Distance (raw segmentation) [mm]:", asd_esophagus)



Heart Dice Coefficient (raw segmentation) : 0.8252127939466718
Heart Hausdorff Distance (raw segmentation) [mm]: 12.700999975204468
Heart Average Surface Distance (raw segmentation) [mm]: 2.9681121353266513

Esophagus Dice Coefficient (raw segmentation) : 0.726747077841333
Esophagus Hausdorff Distance (raw segmentation) [mm]: 84.63958609029844
Esophagus Average Surface Distance (raw segmentation) [mm]: 1.016668111527261


## Exercises, next steps, getting help, references

* I want to train my network, not run inference - what do I do?